In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import pywt
import random


In [16]:
l=5
S=4
ci = [0.11,0.17,0.25,0.25,0.23]
threshold = 0.2 # 0.2 for online dataset, 0.3 - 0.4 for demo


In [3]:
#calculate the wavelet energy feature of ROI

def getE(dir):

    testImg = cv2.imread(dir)
    testImg = cv2.cvtColor(testImg, cv2.COLOR_BGR2GRAY)
    testImg = cv2.resize(testImg,(256,256))
    coeffs =pywt.wavedec2(testImg, 'Haar', mode='per', level = l)
    cA5,(cH5, cV5, cD5), (cH4, cV4, cD4), (cH3, cV3, cD3), (cH2, cV2, cD2), (cH1, cV1, cD1) = coeffs
    HVDs = [(cH1, cV1, cD1), (cH2, cV2, cD2), (cH3, cV3, cD3), (cH4, cV4, cD4),(cH5, cV5, cD5)]
    V=[]
    for HVD in HVDs:
        Vl=[]
        Hf,Vf,Df = HVD
        M,N = Hf.shape
        A = int(M/S)
        subHf = Hf[0:M-M%A, 0:N-N%A].reshape(M//A, A, -1, A).swapaxes(1, 2).reshape(-1, A, A)
        subVf = Vf[0:M-M%A, 0:N-N%A].reshape(M//A, A, -1, A).swapaxes(1, 2).reshape(-1, A, A)
        subDf = Df[0:M-M%A, 0:N-N%A].reshape(M//A, A, -1, A).swapaxes(1, 2).reshape(-1, A, A)
        for h,v,d in zip(subHf, subVf,subDf):
            Eh = np.sum(h**2)
            Ev = np.sum(v**2)
            Ed = np.sum(d**2)
            Vs = [Eh, Ev, Ed]
            Vl.append(Vs)
        V.append(Vl)
    return V

In [17]:
def dist(V1,V2,l,ci):
    V1 = V1 / np.sum(V1)
    V2 = V2 / np.sum(V2)
    temp2 = [0,0,0]
    d=[0,0,0]
    for c, v1,v2 in zip(ci,V1,V2):
        for v12, v22 in zip(v1, v2):
            temp = abs(np.subtract(v12,v22))
            temp2 = np.add(temp,temp2)
        d = np.add(np.dot(c,temp2),d)
    return d

In [18]:
def classifier(V1,V2,l,ci, threshold):
    d = dist(V1,V2,l,ci)
    if(sum(d)>threshold):
        return 0
    elif(sum(d)<=threshold):
        return 1

In [33]:
thresholds = np.arange(0.1,0.32,0.02)
r =random.randrange(1,6000) 
# print(thresholds)
print(r)

3535


In [37]:
print(r//10 *10)

3530


In [31]:

def get_far(labelFalse_predictTrue,labelTrue_predictTrue):
    far = labelFalse_predictTrue/(labelFalse_predictTrue+labelTrue_predictTrue)
    return(far)
 
def get_frr(labelTrue_predictFalse,labelFalse_predictFalse):
    frr = labelTrue_predictFalse/(labelTrue_predictFalse+labelFalse_predictFalse)
    return(frr)


In [38]:
Vr = getE('./images/roi/session1/{0:05}.bmp'.format(r))

In [ ]:
for t in thresholds:
    lf_pf = 0
    lf_pt = 0
    lt_pf = 0
    lt_pt = 0
#     for i in range(len(datas)):
    for j,k in zip(["{0:05}".format(i) for i in range(1,6001)],range(6000)):
        Vtest = getE('./images/roi/session2/'+str(j)+'.bmp')
        pre = classifier(Vr,Vtest,l,ci, t)
#         print(pre,label)
        if((j<(r//10 *10) or j>(r//10 *10+10) ) and pre==0):
            lf_pf += 1
        elif((j<(r//10 *10) or j>(r//10 *10+10)) and pre==1):
            lf_pt += 1
        elif((j>(r//10 *10) and j<(r//10 *10+10)) and pre==0):
            lt_pf += 1
        elif((j>(r//10 *10) and j<(r//10 *10+10)) and pre==1):
            lt_pt += 1
    far = get_far(lf_pt,lt_pt)
    frr = get_frr(lt_pf,lf_pf)
    scores.append([t,far,frr,abs(far-frr)])
scores = np.array(scores)
print(np.array(scores))
 
plt.plot(scores[:,0],scores[:,1])
plt.plot(scores[:,0],scores[:,2])
#plt.plot(scores[:,0],scores[:,3])
plt.show()

In [19]:
#compare tommy1 with stored 9 tommy's roi

V_tommy1 = getE('./images/roi/tommy/tommy1.jpg')
for i in range(1,10):
    Vtest = getE('./images/roi/tommy/test'+str(i)+'.jpg')
#     print(dist(V_tommy1, Vtest,l,ci))
    print(sum(dist(V_tommy1, Vtest,l,ci)))

0.211064636567827
0.20165881811992292
0.20596235420527262
0.20891392621553218
0.18750665504242728
0.20258006680179258
0.19783255477914136
0.20291761166618216
0.19681016987329386


In [20]:
#compare tommy2 with stored 9 tommy's roi


V_tommy2 = getE('./images/roi/tommy/tommy2.jpg')
for i in range(1,10):
    Vtest = getE('./images/roi/tommy/test'+str(i)+'.jpg')
#     print(dist(V_tommy2, Vtest,l,ci))
    print(sum(dist(V_tommy2, Vtest,l,ci)))


0.36057968153097825
0.35383921931049833
0.3696663919094402
0.3570455433807344
0.337677515194567
0.35719929067954276
0.3609195123819375
0.37259842740957694
0.359574134746077


In [21]:
#compare tommy3 (95px * 95px) with stored 9 tommy's roi


V_tommy3 = getE('./images/roi/tommy/tommy3.jpg')
for i in range(1,10):
    Vtest = getE('./images/roi/tommy/test'+str(i)+'.jpg')
#     print(dist(V_tommy3, Vtest,l,ci))
    print(sum(dist(V_tommy3, Vtest,l,ci)))

0.2842737845305574
0.26871255559804824
0.28312364069772655
0.23344709528992363
0.22698238062976045
0.261398268360224
0.26388034228517987
0.2839460995711548
0.273575572983223


In [22]:
#compare eddy1 with stored 9 tommy's roi

V_eddy1 = getE('./images/roi/eddy/eddy1.jpg')
for i in range(1,10):
    Vtest = getE('./images/roi/tommy/test'+str(i)+'.jpg')
#     print(dist(V_eddy1, Vtest,l,ci))
    print(sum(dist(V_eddy1, Vtest,l,ci)))


0.4028936533777637
0.3961280262506939
0.40303349299341923
0.3775465195766488
0.37522163448661316
0.39100534478282867
0.4080058910608747
0.41109651862088414
0.4095910651641321


In [25]:
V_s1_1 = getE('./images/roi/session1/00151.bmp')
record =[]
for i in ["{0:04}".format(i) for i in range(100)]:
    offset = random.randrange(1,9) 
    Vtest = getE('./images/roi/session2/'+str(i)+str(offset)+'.bmp')
#     Vtest2 = getE('./images/roi/session2/'+str(i)+str(offset)+'.bmp')
#     Vtest3 = getE('./images/roi/session2/'+str(i)+str(offset-1)+'.bmp')
#     Vtest = Vtest1 + Vtest2 + Vtest3
#     Vtest = Vtest/3
    record.append(sum(dist(V_s1_1, Vtest,l,ci)))
print(min(record))
print(max(record)/100)
print(record.index(min(record)))

0.11526725616913543
0.0033018652613423217
15


In [10]:
record =[]
for i in ["{0:05}".format(i) for i in range(151,161)]:
#     print(i)
    offset = random.randrange(10) 
    Vtest = getE('./images/roi/session2/'+i+'.bmp')
    record.append(sum(dist(V_s1_1, Vtest,l,ci)))

In [11]:
print(record)

[0.11666201262980237, 0.12285192460850801, 0.14603317481693093, 0.09866831463014436, 0.11526725616913543, 0.11130669736898237, 0.11459738861446586, 0.16845677832035683, 0.11792073066685561, 0.11583011880166943]


In [14]:
counter = 0
for i in record:
    if i < 0.2:
        counter+=1
print(counter)

3
